In [1]:
%matplotlib qt4
from __future__ import division

from models import tools, optimize, models, filters
from models.tests import PerformanceTest

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# The Naive Way

In [29]:
data = tools.load_data(limit=40000, offset=2400000)

Loaded 29981 answers.


In [19]:
grad = optimize.NaiveDescent(data)

In [20]:
descent1 = grad.search_pfa(1.5, -2, step_size=3, maxiter=100, precision=0.005)

gamma: 3.18298528419; grad: -0.000741008113952
delta: -0.303539220688; grad: -0.000729927486385


In [46]:
descent2 = grad.search_pfa(5, 0.5, step_size=2.5, maxiter=100, precision=0.005)

gamma: 3.80556357186; grad: 0.000890172684958
delta: -0.678034006194; grad: 0.000878019898582


In [93]:
descent3 = grad.search_pfag(1.5, -2, step_size=20, maxiter=36, precision=0.005)

gamma: 1.85300525851; grad: 0.000260020806278
delta: -1.57543851455; grad: 0.000284481491125


In [37]:
elo = models.EloModel()
pfa = models.PFAModel(elo, gamma=2.99622612646, delta=-0.476090204636)
pfa_test = PerformanceTest(pfa, data)
pfa_test.run()

In [38]:
pfa_test.results['train']

RMSE: 0.361919204016
AUC: 0.787337458274
OFF: -6.24923069748e-05
Set Size: 50000

In [120]:
def annotate(descent, number, mark, xadd, yadd):
    row = descent.params.loc[number]
    grad = descent.grads.loc[number]
    plt.annotate(r'$\gamma={}$, $\delta={}$'.format(round(row.gamma, 2), round(row.delta, 2)),
                 xy=(number, grad), xycoords='data',
                 xytext=(number + xadd, grad + yadd), textcoords='data',
                 bbox=dict(boxstyle="round", fc="w"))
    plt.plot(number, grad, mark)
    
#annotate(descent1, 1, 'go', 0.8, -0.006)
#annotate(descent1, 10, 'go', 0.8, -0.006)
annotate(descent1, 34, 'go', -8, -0.009)

#annotate(descent3, 1, 'ro', 0.7, 0.004)
#annotate(descent3, 11, 'ro', 0.8, 0.004)
annotate(descent3, 20, 'ro', 0.8, 0.006)

plt.xlabel('number of iteration')
plt.ylabel('predicted - observed')

plt.xlim([0, 35])
plt.ylim([-0.08, 0.03])

line1, = plt.plot(descent1.grads[:35], 'g', label=r'step size = $3$')
line2, = plt.plot(descent3.grads[:36], 'r', label=r'step size = $20$')

plt.legend(handles=[line1, line2], loc='lower right')

plt.show()

# The Proper Way

In [68]:
reload(optimize)

<module 'models.optimize' from '/home/pavel/Projects/thesis/models/optimize.pyc'>

In [52]:
data = tools.load_data(limit=100000, offset=2000000)

Loaded 85915 answers.


In [59]:
data = data[filters.for_staircase(data)]

In [64]:
descent = optimize.GradientDescent(data)

In [65]:
result = descent.search_staircase(init_learn_rate=0.01, number_of_iter=15)

   2.50000    0.80000        inf
   2.53135    1.07563   -0.00354
   2.58847    1.12907   -0.00375
   2.58106    1.12488   -0.00435
   2.57004    1.10855   -0.00451
   2.56305    1.08860   -0.00454
   2.55936    1.06860   -0.00454
   2.55767    1.05011   -0.00453
   2.55710    1.03360   -0.00453
   2.55713    1.01907   -0.00452
   2.55747    1.00633   -0.00452
   2.55798    0.99515   -0.00452
   2.55855    0.98532   -0.00452
   2.55916    0.97665   -0.00451
   2.55976    0.96896   -0.00451
   2.56035    0.96213   -0.00451


In [66]:
result

Iterations: 15
Gamma: 2.56035
Delta: 0.96213
Staircase:
       60       90      150      300      600     1800    10800    86400   604800      inf
   +1.259   +0.671   +0.567   +0.200   +0.340   +0.363   +0.286   +0.012   +0.024   +0.000

In [45]:
result.plot(color='orange')

In [3]:
train_data = []
for i in range(10):
    limit, offset = 120000, i * 120000 + 1000000
    df = tools.load_data(limit=limit, offset=offset, echo_loaded=False)
    df = df[filters.for_staircase(df)]
    train_data.append(df.copy())
    tools.echo('[{}]: Loaded {} answers.'.format(i, len(df)), clear=False)

tools.echo('Data loaded.')

results = []
for i in range(10):
    descent = optimize.GradientDescent(train_data[i])
    res = descent.search_staircase(init_learn_rate=0.01, number_of_iter=25,
                                   echo_iterations=False)
    results.append(res)
    tools.echo('[{}]: done!'.format(i), clear=False)

[9]: done!


In [5]:
x_matrix = []
y_matrix = []
for res in results:
    stairs = sorted(res.staircases[-1].items(), key=lambda x: x[0])
    staircase_times = res.model.metadata['staircase_times']

    xi_axis = [np.mean(staircase_times[i]) for i in res.intervals]
    yi_axis = [value for interval, value in stairs]
    
    x_matrix.append(xi_axis)
    y_matrix.append(yi_axis)

x_axis = []
y_axis = []
e_vals = []
for i in range(len(x_matrix[0])):
    x_axis += [np.mean([x_matrix[j][i] for j in range(len(x_matrix))])]
    y_axis += [np.mean([y_matrix[j][i] for j in range(len(x_matrix))])]
    e_vals += [np.std([y_matrix[j][i] for j in range(len(x_matrix))]) / 2]

plt.errorbar(x_axis, y_axis, e_vals,
             ecolor='orange', elinewidth='2',
             linestyle='--', linewidth='2',
             capthick='2', capsize=4,
             color='#02A5F4', marker='o')

plt.xscale('log')
plt.xlabel('Time from previous attempt in seconds.')
plt.ylabel('Memory activation')

plt.show()

In [8]:
gamma_std = np.mean([res.gammas[-1] for res in results])
delta_std = np.mean([res.deltas[-1] for res in results])
tools.echo('gamma={:.3f}, delta={:.3f}'.format(gamma_std, delta_std), clear=False)

gamma=2.281, delta=0.865
